<a href="https://colab.research.google.com/github/Luensmann/Bachelorarbeit/blob/main/Trainer/tmvar_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate
!pip install wandb
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Features, Sequence, Value, ClassLabel
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, Trainer, DataCollatorForTokenClassification
import evaluate
import json
import wandb
import os
import numpy as np


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# labels

labels = ['O', 'B-DNAMutation', 'I-DNAMutation', 'B-ProteinMutation', 'I-ProteinMutation', 'B-SNP', 'I-SNP']

In [ ]:
# Load Dataset
dataset = load_dataset("Brizape/tmvar_tokenized_split_0404_dev")
dataset = dataset.cast_column("ner_tags", Sequence(feature=ClassLabel(names=labels)))
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/201 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/801 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Brizape___parquet/Brizape--tmvar_tokenized_split_0404_dev-3b41f35c1b43fb07/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Casting the dataset:   0%|          | 0/201 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/498 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/801 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 201
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 498
    })
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 801
    })
})

In [ ]:
tokenized_dataset = dataset.remove_columns(["id", "tokens", "ner_tags", "texts"])

In [ ]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-DNAMutation', 'I-DNAMutation', 'B-ProteinMutation', 'I-ProteinMutation', 'B-SNP', 'I-SNP'], id=None), length=-1, id=None),
 'texts': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [ ]:
# Tokenizer
tokenizer_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trunction=True , model_max_len=512)

In [ ]:
# padding
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
# metric
seqeval = evaluate.load("seqeval")

In [ ]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-DNAMutation',
 'I-DNAMutation',
 'B-ProteinMutation',
 'I-ProteinMutation',
 'B-SNP',
 'I-SNP']

In [ ]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

import numpy as np

labels = labels


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels, mode='strict')
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {0: 'O',
 1: 'B-DNAMutation',
 2: 'I-DNAMutation',
 3: 'B-ProteinMutation',
 4: 'I-ProteinMutation',
 5: 'B-SNP',
 6: 'I-SNP'}
label2id = {'B-DNAMutation': 1,
 'B-ProteinMutation': 3,
 'B-SNP': 5,
 'I-DNAMutation': 2,
 'I-ProteinMutation': 4,
 'I-SNP': 6,
 'O': 0}

In [ ]:
%env WANDB_PROJECT=all_0504

env: WANDB_PROJECT=all_0504


In [ ]:
# Training several model with different learning rates
ln_rate = [1e-5, 2e-5, 1e-4]

for learnrate in ln_rate:

  runname = "tmvar_" + str(learnrate)

  model = AutoModelForTokenClassification.from_pretrained(
      "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", num_labels=7, id2label=id2label, label2id=label2id
  )

  training_args = TrainingArguments(
      report_to = 'wandb',                     # enable logging to W&B
      output_dir = runname,    # output directory/ name for huggingface hub
      learning_rate=learnrate,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      weight_decay=0.01,
      evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
      max_steps = 2000,
      logging_steps = 25,                    # we will log every 25 steps
      eval_steps = 25,                      # we will perform evaluation every 25 steps
      save_steps = 25,
      load_best_model_at_end=True,
      metric_for_best_model = 'eval_loss',
      greater_is_better = False,
      push_to_hub=True,
      run_name = runname             # name of the W&B run
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset["train"],
      eval_dataset=tokenized_dataset["validation"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
      callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
  )

  trainer.train()
  wandb.finish()

  trainer.save_model()

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForToken

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
25,1.030300,0.265215,0.000000,0.000000,0.000000,0.955530
50,0.230300,0.230779,0.000000,0.000000,0.000000,0.955530
75,0.155700,0.104271,0.000000,0.000000,0.000000,0.965168
100,0.087000,0.072419,0.000000,0.000000,0.000000,0.979110
125,0.064600,0.061381,0.000000,0.000000,0.000000,0.981575
150,0.048400,0.052595,0.000000,0.000000,0.000000,0.982965
175,0.035000,0.044798,0.694444,0.126904,0.214592,0.984489
200,0.033100,0.037645,0.824561,0.477157,0.604502,0.987986
225,0.019700,0.041433,0.764045,0.690355,0.725333,0.988389
250,0.015900,0.033322,0.853403,0.827411,0.840206,0.991751


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-d

eval/accuracy,▁▁▃▆▆▆▇▇▇███
eval/f1,▁▁▁▁▁▁▃▆▇███
eval/loss,█▇▃▂▂▂▁▁▁▁▁▁
eval/precision,▁▁▁▁▁▁▇█▇█▇█
eval/recall,▁▁▁▁▁▁▂▅▇███
eval/runtime,▁▂▃▅▄█▆▅▆▇▆▆
eval/samples_per_second,█▇▆▄▄▁▃▄▃▂▃▂
eval/steps_per_second,█▇▆▄▄▁▃▄▃▂▃▂
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
train/learning_rate,█▇▇▆▅▅▄▄▃▂▂▁


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/Brizape/tmvar_1e-05_0404_ES6_strict_tok1
   a697e0b..3abb5ef  main -> main

   a697e0b..3abb5ef  main -> main



In [ ]:
wandb.finish()
trainer.save_model()

eval/accuracy,▁▁▇▇███
eval/f1,▁▂▇██▇█
eval/loss,█▃▂▁▁▂▁
eval/precision,▁▆█▇█▇█
eval/recall,▁▁▆█▇▇▇
eval/runtime,▁▃▄▆▇█▆
eval/samples_per_second,█▆▅▃▂▁▃
eval/steps_per_second,█▆▅▃▂▁▃
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/learning_rate,█▇▆▅▃▂▁


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/Brizape/SETH_5e-05_0404_ES6_strict_tok
   b4087f7..42d5266  main -> main

   b4087f7..42d5266  main -> main

